# (Part. 1)

In [31]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 6]

from ipywidgets import interact
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.palettes import Category20
from bokeh.models import Label
colors = Category20[20]
output_notebook()

from risk_functions import Simulate_Price_Black_Scholes

Loading BokehJS ...

# Définition des paramètres

In [32]:
Q = 10000 # Quantité produite
K = 40 # Cout de production
initial_value = 150 # S_{t_0} Prix de vente d'une unité à l'instant initial.
tendency = 0.05 # mu / tendance
volatility = 0.2 # sigma / volatilité
nb_simulations = 10000

# Simulation des prix de vente

In [33]:
# Simulation des prix: Par défaut les prix sont simulés sur une période d'un an, avec un pas de simulation d'un jour.
simulated_prices = Simulate_Price_Black_Scholes(initial_value, tendency, volatility, nb_simulations)

# Analyse des trajectoires simulées

In [34]:
# Plot des 100 premières simulation de prix
trajectory_plot = figure(title='Simulated Price Trajectories using Black-Scholes Model', x_axis_label='Time Steps', y_axis_label='Price', sizing_mode="stretch_width", aspect_ratio=10/6)

plots = []
for i in range(min(nb_simulations, 100)):
    plots.append(trajectory_plot.line(np.arange(len(simulated_prices[i])), simulated_prices[i], color=colors[i % len(colors)]))
show(trajectory_plot, notebook_handle=True)


# Code utile pour l'affichage dynamique
def update_trajectory(initial_value, tendency, volatility):
    simulated_prices = Simulate_Price_Black_Scholes(initial_value, tendency, volatility, nb_simulations)
    for idx, r in enumerate(plots):
        r.data_source.data['y'] = simulated_prices[idx]
    # Estimation de la tendance (drift ou mu) et de la volatilité (sigma) à partir des trajectoires
    print("Tendance empirique:", np.mean(np.log(simulated_prices[:,-1]/simulated_prices[:,0]))) # Peut-etre comparé à la valeur de la tendance
    print("Volatilité empirique:",np.std(np.log(simulated_prices[:,-1]/i))) # Peut-etre comparé à la valeur de la volatilité
    push_notebook()

interact(update_trajectory, initial_value=(0,300,10), tendency=(-1,2,0.01), volatility=(0.01,1,0.01))

interactive(children=(IntSlider(value=150, description='initial_value', max=300, step=10), FloatSlider(value=0…

<function __main__.update_trajectory(initial_value, tendency, volatility)>

# On affiche l'histogramme des prix à un an

In [35]:
fig = figure(title='Histogram of $$S_T$$', x_axis_label='Value of $$S_T$$', y_axis_label='Frequency', sizing_mode="stretch_width", aspect_ratio=10/6)
S_T = simulated_prices[:,-1] # S_T est le prix final, i.e, le prix dans un an

# Histogram
hist, edges = np.histogram(S_T, density=True, bins=50)
ST_hist = fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="black")

show(fig, notebook_handle=True)


# Code utile pour l'affichage dynamique
def update_hist_ST(initial_value, tendency, volatility):
    simulated_prices = Simulate_Price_Black_Scholes(initial_value, tendency, volatility, nb_simulations)
    S_T = simulated_prices[:, -1]
    hist, edges = np.histogram(S_T, density=True, bins=50)
    ST_hist.data_source.data.update({
        'top': hist,
        'left': edges[:-1],
        'right': edges[1:]
    })
    push_notebook()

interact(update_hist_ST, initial_value=(0,300,10), tendency=(-1,2,0.01), volatility=(0.01,1,0.01))

interactive(children=(IntSlider(value=150, description='initial_value', max=300, step=10), FloatSlider(value=0…

<function __main__.update_hist_ST(initial_value, tendency, volatility)>

# L'histogramme des cashflows

In [36]:
cash_flows = Q*(S_T - K) # Les cashflows finaux sont obtenus par : "quantité * (Prix de vente - Cout de production)"
fig = figure(title='Histogram of Cash Flows', x_axis_label='cash_flow', y_axis_label='Frequency', sizing_mode="stretch_width", aspect_ratio=10/6)

# Histogram
hist, edges = np.histogram(cash_flows, density=True, bins=50)
cash_flows_hist = fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="black")

show(fig, notebook_handle=True)

def update_hist_cash_flows(initial_value, tendency, volatility, prod_cost, volume):
    simulated_prices = Simulate_Price_Black_Scholes(initial_value, tendency, volatility, nb_simulations)
    S_T = simulated_prices[:,-1]
    cash_flows = volume*(S_T - prod_cost)
    
    hist, edges = np.histogram(cash_flows, density=True, bins=50)
    
    cash_flows_hist.data_source.data['top'] = hist
    cash_flows_hist.data_source.data['left'] = edges[:-1]
    cash_flows_hist.data_source.data['right'] = edges[1:]
    
    push_notebook()

interact(update_hist_cash_flows, initial_value=(0,300,10), tendency=(-1,2,0.01), volatility=(0.01,1,0.01), prod_cost=(0.1,200,10), volume=(1,200,10))

interactive(children=(IntSlider(value=150, description='initial_value', max=300, step=10), FloatSlider(value=0…

<function __main__.update_hist_cash_flows(initial_value, tendency, volatility, prod_cost, volume)>

# (Part. 2) On veut maintenant calculer les statistiques suivantes sur nos cashflows :
(Les statistiques sont calculées sur les valeurs initiales du début de TP)
* L'écart-type
* Les quantiles 1%, 5%, 10%
* Les EaRs

In [37]:
# Ecart-type
print("ECART-TYPES")
std_dev = np.std(cash_flows)
print("  Ecart-type des cash flows:", std_dev)

# Quantiles
print("\nQUANTILES")
quantiles = [1, 5, 10]
quantile_values = np.percentile(cash_flows, quantiles)
print(f"  Quantiles à {quantiles}%:", quantile_values)

# EaRs
print("\nEARS")
mean_cash_flows = np.mean(cash_flows)
EaR_quantile = ['99%', '95%', '90%']
EaRs = mean_cash_flows - quantile_values
for q, ear in zip(EaR_quantile, EaRs):
    print(f"  EaR {q}:", ear)

ECART-TYPES
  Ecart-type des cash flows: 318365.07164483843

QUANTILES
  Quantiles à [1, 5, 10]%: [576187.74174867 709969.83955504 792199.75802338]

EARS
  EaR 99%: 597938.8107356792
  EaR 95%: 464156.71292931226
  EaR 90%: 381926.7944609723


# On génère à nouveau l'histogramme des cashflows en ajoutant les statistiques précédemments calculées

In [41]:
cash_flows = Q*np.maximum(S_T - K,0) # Les cashflows finaux sont obtenus par : "quantité * (Prix de vente - Cout de production)"
fig = figure(title='Histogram of Cash Flows', x_axis_label='Cash Flow', y_axis_label='Frequency', sizing_mode="stretch_width", aspect_ratio=10/6)

# Histogram
hist, edges = np.histogram(cash_flows, density=True, bins=50)
cash_flows_hist = fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="black")


quantile_lines = []

# Ajoutez des lignes verticales pour les quantiles
quantile_values = np.percentile(cash_flows, quantiles)
for i, q_val in enumerate(quantile_values):
    quantile_lines.append(fig.line([q_val, q_val], [0, max(hist)], line_color='blue', line_dash='dashed', line_width=2, legend_label=f'{quantiles[i]}% Quantile'))

show(fig, notebook_handle=True)

def update_stats(initial_value, tendency, volatility, prod_cost, volume):
    simulated_prices = Simulate_Price_Black_Scholes(initial_value, tendency, volatility, nb_simulations)
    S_T = simulated_prices[:,-1]
    cash_flows = volume*np.maximum(S_T - prod_cost,0)
    
    hist, edges = np.histogram(cash_flows, density=True, bins=50)
    
    cash_flows_hist.data_source.data['top'] = hist
    cash_flows_hist.data_source.data['left'] = edges[:-1]
    cash_flows_hist.data_source.data['right'] = edges[1:]

    #Update stat lines
    quantile_values = np.percentile(cash_flows, quantiles)

    for idx, quantile_line in enumerate(quantile_lines):
        quantile_line.data_source.data['x'] = [quantile_values[idx], quantile_values[idx]]
        quantile_line.data_source.data['y'] = [0, max(hist)]

    push_notebook()

interact(update_stats, initial_value=(0,300,10), tendency=(-1,2,0.01), volatility=(0.01,1,0.01), prod_cost=(0.1,200,10), volume=(1,200,10))

interactive(children=(IntSlider(value=150, description='initial_value', max=300, step=10), FloatSlider(value=0…

<function __main__.update_stats(initial_value, tendency, volatility, prod_cost, volume)>